In [1]:
import os
import pandas as pd
from datetime import datetime
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import logging
from utils.logging_setup import logging_setup

In [2]:
# Parameters
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
log_name = "markets_exe"
ssaa_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/raw/elec/ssaa_c2"
ssaa_path_df = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/ssaa_c2.pkl"
ssaa_path_df_dt = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/ssaa_c2_detailed.pkl"


In [3]:
ssaa_apagon_25 = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/raw/elec/blackout_apr_2025.xlsx"

In [4]:
logging_setup(log_name, config_path)

In [5]:
try:
    df_ssaa = pd.read_pickle(ssaa_path_df)
    start_date = df_ssaa.iloc[-1, 0]
    del df_ssaa
except:
    start_date = datetime(2023,1,1)


In [6]:
import shared.ssaa_ree as ssaa_ree

In [8]:
LiquiComun = ssaa_ree.Liquicomon(
                                start_date=start_date,
                                end_date=datetime.now(),
                                extract_path=ssaa_path
                                )

In [9]:
logging.info(f"Procesando archivos en el directorio {ssaa_path}")
LiquiComun.ssaa()

2025-08-17 12:13:14,690 - INFO - Procesando archivos en el directorio /Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/raw/elec/ssaa_c2


In [10]:
from utils.utils_var import set_datetime_h

logging.info("Añadiendo liquidacón de los días 28 y 29 de abril de 2025 (blackout)...")
df_ssaa = LiquiComun.df_ssaa

df_blackout = pd.read_excel(ssaa_apagon_25)
df_blackout.columns = ['datetime', 'hour', 'ssaa']
df_blackout = set_datetime_h(date='datetime', hour='hour', load='ssaa', df=df_blackout)
df_blackout.columns = ['datetime', 'ssaa']
df_blackout

df_ssaa = pd.concat([df_ssaa, df_blackout], ignore_index=True)
df_ssaa = df_ssaa.sort_values(by='datetime').reset_index(drop=True)

2025-08-17 12:13:21,239 - INFO - Añadiendo liquidacón de los días 29 y 29 de abril de 2025...


,datetime,month,year,ssaa
0,2018-02-01 00:00:00,2.0,2018.0,3.516075
1,2018-02-01 01:00:00,2.0,2018.0,3.529241
2,2018-02-01 02:00:00,2.0,2018.0,3.550439
3,2018-02-01 03:00:00,2.0,2018.0,3.984606
4,2018-02-01 04:00:00,2.0,2018.0,4.211614
...,...,...,...,...
64266,2025-05-31 19:00:00,5.0,2025.0,17.172741
64267,2025-05-31 20:00:00,5.0,2025.0,15.426091
64268,2025-05-31 21:00:00,5.0,2025.0,16.026816
64269,2025-05-31 22:00:00,5.0,2025.0,14.124158


In [11]:
df_ssaa.to_pickle(ssaa_path_df)
logging.info("Liquidación de servicios de ajuste completada y guardada en el archivo ssaa_c2.pkl")

2025-08-17 12:13:21,312 - INFO - Liquidación de servicios de ajuste completada y guardada en el archivo ssaa_c2.pkl


In [12]:
logging.info(f"Procesando archivos en el directorio {ssaa_path}, con cada concepto por separado...")

LiquiComun.ssaa_desgolsado();

2025-08-17 12:13:21,317 - INFO - Procesando archivos en el directorio /Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/raw/elec/ssaa_c2, con cada concepto por separado...


In [13]:
df_ssaa_dt = LiquiComun.df_ssaa_dt
df_ssaa_dt.to_pickle(ssaa_path_df_dt)
logging.info("Liquidación de servicios de ajuste detallada completada y guardada en el archivo ssaa_c2_detailed.pkl.\n")

2025-08-17 12:13:39,363 - INFO - Liquidación de servicios de ajuste detallada completada y guardada en el archivo ssaa_c2_detailed.pkl.

